These scripts parses text looking for ability and skill checks and their DC.

In [2]:
import re, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
class AdventureParser:

    skill_pattern = re.compile(r"\bDC\s+(\b\d+\b)\s+\b(Strength|Dexterity|Constitution|Wisdom|Intelligence|Charisma)\s+\((.+?)\)")
    save_pattern = re.compile(r"\bDC\s+(\b\d+\b)\s+\b(Strength|Dexterity|Constitution|Wisdom|Intelligence|Charisma)\s+?saving throw\b")

    def __init__(self, filename, adventure_name):
        self.adventure_name = adventure_name
        self.filename = filename
        self.skill_checks = []
        self.saving_throws = []
        self.skill_data = {}
        self.save_data = {}
        self.find_skill_checks()
        self.find_saving_throws()
        self.skill_data = self.prepare_data(self.skill_checks)
        self.save_data = self.prepare_data(self.saving_throws)

    def find_skill_checks(self):
        text = ""
        with open(self.filename, "r", encoding="utf-8") as textfile:
            text = ' '.join(textfile.readlines()).replace('\n', '')
        for match in self.skill_pattern.finditer(text):
            d = {
                'DC': int(match.group(1)),
                'Ability': match.group(2),
            }
            try:
                d['Skill'] = match.group(3)
            except IndexError:
                d['Skill'] = None
            self.skill_checks.append(d)

    def find_saving_throws(self):
        text = ""
        with open(self.filename, "r", encoding="utf-8") as textfile:
            text = ' '.join(textfile.readlines()).replace('\n', '')
        for match in self.save_pattern.finditer(text):
            d = {
                'DC': int(match.group(1)),
                'Ability': match.group(2),
            }
            self.saveing_throws.append(d)
    
    def prepare_data(data):
        dc_set = sorted(list(set([item['DC'] for item in data])))
        ability_set = sorted(list(set([item['Ability'] for item in data])))

        dc_count = {}
        for dc in dc_set:
            dc_count[dc] = [item['DC'] for item in data].count(dc)

        ability_count = {}
        for ability in ability_set:
            ability_count[ability] = [item['Ability'] for item in data].count(ability)

        ability_data = {}
        for ability in ability_set:
            ability_data[ability] = [item['DC'] for item in data if item['Ability'] == ability]
        
        return {
            'dc_set': dc_set,
            'ability_set': ability_set,
            'dc_count': dc_count, 
            'ability_count': ability_count, 
            'ability_data': ability_data}

In [7]:
# with open('dnd_data/rawmodules/wbtw_skills.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['Ability',] + skill_data['dc_set'])
#     for ability in skill_data['ability_set']:
#         ln = [ability,] + [skill_data['ability_data'][ability].count(dc) for dc in skill_data['dc_set']]
#         writer.writerow(ln)

# with open('dnd_data/rawmodules/wbtw_saves.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['Ability',] + save_data['dc_set'])
#     for ability in save_data['ability_set']:
#         ln = [ability,] + [save_data['ability_data'][ability].count(dc) for dc in save_data['dc_set']]
#         writer.writerow(ln)


In [10]:
with open('dnd_data/rawmodules/wbtw_save_totals.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Ability', 'Count'] )
    for k, v in save_data['ability_data'].items():
        writer.writerow([k, len(v)])